In [2]:
from google.colab import drive
import os
import cv2
import numpy as np
import albumentations as A

# Mount Google Drive to access your dataset
drive.mount('/content/drive')

# Install any missing dependencies
!pip install albumentations opencv-python-headless numpy


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Path to the shortcut folder in your Google Drive
BASE_DIR = "/content/drive/My Drive/FYP_work/DecayDataSrc"

# Test if the shortcut folder is accessible
import os

print("DecayDataSrc directory exists:", os.path.exists(BASE_DIR))
print("Contents of DecayDataSrc:", os.listdir(BASE_DIR) if os.path.exists(BASE_DIR) else "Not found!")


Mounted at /content/drive
DecayDataSrc directory exists: True
Contents of DecayDataSrc: ['src', 'Teeth_Dataset', 'Decay_Dataset']


In [4]:
import os
import cv2

def create_directory(path_segments):
    """Creates a directory if it does not exist."""
    path = os.path.join(*path_segments)
    # Adding a check to see if path exists and if it's writable
    if not os.path.exists(path):
        try:
            os.makedirs(path, exist_ok=True)
        except OSError as e:
            if e.errno == 30:  # Read-only file system error
                print(f"Error: Cannot create directory '{path}'. File system is read-only.")
                # You might want to handle this error differently, e.g., by changing the path or using a different storage location.
            else:
                raise  # Re-raise other errors
    # If path exists and is a directory, no need to create it
    elif os.path.isdir(path):
        print(f"Directory '{path}' already exists. Skipping creation.")
    else:
        print(f"Error: '{path}' exists but is not a directory.")
    return path


def interpolate_image(image, target_size):
    """Interpolates an image to the target size."""
    # ... (Your interpolation logic using cv2 or other libraries) ...
    # Example using cv2.resize:
    return cv2.resize(image, target_size)

BASE_DIR = "/content/drive/My Drive/FYP_work/DecayDataSrc/Decay_Dataset"

train_images_dir = f"{BASE_DIR}/Train_Data/Images"
train_masks_dir = f"{BASE_DIR}/Train_Data/Masks"
test_images_dir = f"{BASE_DIR}/Test_Data/Images"
test_masks_dir = f"{BASE_DIR}/Test_Data/Masks"

# Output directories
augmented_images_dir = create_directory([BASE_DIR, "augmented/Images"])
augmented_masks_dir = create_directory([BASE_DIR, "augmented/Masks"])
compressed_dir = create_directory([BASE_DIR, "compressed"])

In [5]:
import os

# Check existence of directories
print("Train images directory exists:", os.path.exists(train_images_dir))
print("Train masks directory exists:", os.path.exists(train_masks_dir))

# List files
if os.path.exists(train_images_dir):
    print("Train Images:", os.listdir(train_images_dir)[:5])  # Show first 5 images
if os.path.exists(train_masks_dir):
    print("Train Masks:", os.listdir(train_masks_dir)[:5])  # Show first 5 masks


Train images directory exists: True
Train masks directory exists: True
Train Images: ['556.png', '96.png', '524.png', '2.png', '11.png']
Train Masks: ['510.png', '96.png', '576.png', '532.png', '50.png']


In [6]:
print(f"Train images directory path: {train_images_dir}")
print("Directory exists:", os.path.exists(train_images_dir))

Train images directory path: /content/drive/My Drive/FYP_work/DecayDataSrc/Decay_Dataset/Train_Data/Images
Directory exists: True


In [7]:
input_image_size = 224
target_shape = (input_image_size, input_image_size)
# Augmentation transformations
transform_crop = A.Compose([A.RandomCrop(width=input_image_size, height=input_image_size, p=1)])
transform_rotate90 = A.Compose([A.RandomRotate90(p=1)])
transform_flip = A.Compose([A.HorizontalFlip(p=1)])
transform_brightness = A.Compose([A.RandomBrightnessContrast(p=1)])
transform_all = A.Compose([
    A.RandomCrop(width=input_image_size, height=input_image_size, p=1),
    A.HorizontalFlip(p=1),
    A.RandomRotate90(p=1),
    A.RandomBrightnessContrast(p=1)
])

In [8]:
# Augment training data
for file_name in os.listdir(train_images_dir):
    try:
        # Load image and mask
        image = cv2.imread(f"{train_images_dir}/{file_name}", cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(f"{train_masks_dir}/{file_name}", cv2.IMREAD_GRAYSCALE)

        # Normalize mask values
        mask[mask == 255] = 1

        # Save original data
        cv2.imwrite(f"{augmented_images_dir}/00_{file_name}", image)
        #cv2.imwrite(f"{augmented_masks_dir}/00_{file_name}", mask)

        # Apply transformations and save
        transformed = transform_crop(image=image, mask=mask)
        cv2.imwrite(f"{augmented_images_dir}/01_{file_name}", transformed["image"])
        #cv2.imwrite(f"{augmented_masks_dir}/01_{file_name}", transformed["mask"])

        transformed = transform_rotate90(image=image, mask=mask)
        cv2.imwrite(f"{augmented_images_dir}/02_{file_name}", transformed["image"])
        #cv2.imwrite(f"{augmented_masks_dir}/02_{file_name}", transformed["mask"])

        transformed = transform_flip(image=image, mask=mask)
        cv2.imwrite(f"{augmented_images_dir}/03_{file_name}", transformed["image"])
        #cv2.imwrite(f"{augmented_masks_dir}/03_{file_name}", transformed["mask"])

        transformed = transform_all(image=image, mask=mask)
        cv2.imwrite(f"{augmented_images_dir}/04_{file_name}", transformed["image"])
        #cv2.imwrite(f"{augmented_masks_dir}/04_{file_name}", transformed["mask"])
    except Exception as e:
        print(f"Error processing {file_name}: {e}")


Error processing 41.png: 'NoneType' object does not support item assignment


In [14]:
import albumentations as A

# Define transform_noise
transform_noise = A.Compose([
    A.RandomBrightnessContrast(p=1, always_apply=True)
])


In [19]:
for file_name in os.listdir(train_images_dir):
    try:
        # Load image and mask
        image = cv2.imread(f"{train_images_dir}/{file_name}", cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(f"{train_masks_dir}/{file_name}", cv2.IMREAD_GRAYSCALE)

        if image is None or mask is None:
            print(f"Missing file: {file_name}")
            continue

        # Normalize the mask: Convert white (255) to foreground (1), keep background as 0
        mask[mask == 255] = 1

        # Save the original mask
        cv2.imwrite(f"{augmented_masks_dir}/00_{file_name}", (mask * 255).astype("uint8"))

        # Apply augmentations
        transformed = transform_crop(image=image, mask=mask)
        augmented_mask = transformed["mask"]
        cv2.imwrite(f"{augmented_masks_dir}/01_{file_name}", (augmented_mask * 255).astype("uint8"))

        transformed = transform_rotate90(image=image, mask=mask)
        augmented_mask = transformed["mask"]
        cv2.imwrite(f"{augmented_masks_dir}/02_{file_name}", (augmented_mask * 255).astype("uint8"))

        transformed = transform_noise(image=image, mask=mask)
        augmented_mask = transformed["mask"]
        cv2.imwrite(f"{augmented_masks_dir}/03_{file_name}", (augmented_mask * 255).astype("uint8"))

        transformed = transform_all(image=image, mask=mask)
        augmented_mask = transformed["mask"]
        cv2.imwrite(f"{augmented_masks_dir}/04_{file_name}", (augmented_mask * 255).astype("uint8"))

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")


Missing file: 41.png


In [23]:
compressed_dir = f"{BASE_DIR}/compressed"
os.makedirs(compressed_dir, exist_ok=True)

# Save data in .npz format
for file_name in os.listdir(augmented_images_dir):
    try:
        # Load augmented image and mask
        image = cv2.imread(f"{augmented_images_dir}/{file_name}", cv2.IMREAD_GRAYSCALE).astype("float32")
        mask = cv2.imread(f"{augmented_masks_dir}/{file_name}", cv2.IMREAD_GRAYSCALE).astype("uint8")
        mask[mask == 255] = 1  # Normalize mask

        # Resize
        image = cv2.resize(image, (input_image_size, input_image_size))
        mask = cv2.resize(mask, (input_image_size, input_image_size))

        # Save compressed
        np.savez_compressed(f"{compressed_dir}/{file_name}.npz", image=image, mask=mask)
    except Exception as e:
        print(f"Failed to compress {file_name}: {e}")


In [24]:
# Check augmented data
print("Augmented images:", os.listdir(augmented_images_dir)[:5])
print("Augmented masks:", os.listdir(augmented_masks_dir)[:5])

# Check compressed data
print("Compressed files:", os.listdir(compressed_dir)[:5])


Augmented images: ['00_556.png', '01_556.png', '02_556.png', '03_556.png', '04_556.png']
Augmented masks: ['00_556.png', '01_556.png', '02_556.png', '03_556.png', '04_556.png']
Compressed files: ['00_556.png.npz', '01_556.png.npz', '02_556.png.npz', '03_556.png.npz', '04_556.png.npz']
